# Homework Starter — Stage 04: Data Acquisition and Ingestion
Name: Dimash Zhanbyrshy
Date: 08/20/2025

## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [18]:
import os, pathlib, datetime as dt
import requests, time
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

RAW = pathlib.Path('../data/raw'); RAW.mkdir(parents=True, exist_ok=True)
load_dotenv(); print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

ALPHAVANTAGE_API_KEY loaded? False


## Helpers (use or modify)

In [19]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

## Part 1 — API Pull (Required)
Choose an endpoint (e.g., Alpha Vantage or use `yfinance` fallback).

In [20]:
# Choose one ticker or endpoint
BASE_URL = "https://api.exchange.coinbase.com"
GRANULARITY = 3600      # 1h candles
MAX_CANDLES = 300       # per-request limit

def _iso(utc_dt):
    return utc_dt.astimezone(dt.timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

def fetch_candles_coinbase(product_id, start_dt, end_dt, granularity=GRANULARITY, pause_sec=0.25):
    # Request data with requests (or yfinance as fallback)
    rows = []
    step = dt.timedelta(seconds=granularity * MAX_CANDLES)
    t0 = start_dt
    while t0 < end_dt:
        t1 = min(t0 + step, end_dt)
        params = {"start": _iso(t0), "end": _iso(t1), "granularity": granularity}
        r = requests.get(f"{BASE_URL}/products/{product_id}/candles", params=params, timeout=30)
        r.raise_for_status()
        rows.extend(r.json())   # [time, low, high, open, close, volume]
        t0 = t1
        time.sleep(pause_sec)

    # Convert to DataFrame; parse dtypes (dates, floats)
    df = pd.DataFrame(rows, columns=["time","low","high","open","close","volume"]).sort_values("time")
    df["time"] = pd.to_datetime(df["time"], unit="s", utc=True)
    for c in ["low","high","open","close","volume"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # Validate (required columns, NA counts, shape)
    rep = validate(df, required=["time","open","high","low","close","volume"])
    if rep["missing"] or rep["na_total"] > 0 or rep["shape"][0] == 0:
        raise ValueError(f"Validation failed: {rep}")
    return df.reset_index(drop=True)

# choose symbols + window
pairs = ["BTC-USD", "ETH-USD"]
end_dt = dt.datetime.now(dt.timezone.utc).replace(minute=0, second=0, microsecond=0)
start_dt = end_dt - dt.timedelta(days=90)

dfs = {}
for pid in pairs:
    df_api = fetch_candles_coinbase(pid, start_dt, end_dt, GRANULARITY)
    window = f"{start_dt:%Y%m%d}_to_{end_dt:%Y%m%d}"
    # Save raw CSV to data/raw/
    _ = save_csv(
        df_api.sort_values("time"),
        prefix="api",
        source="coinbase",
        symbol=pid.replace("-", ""),
        window=window,
        granularity="1h"
    )
    dfs[pid] = df_api

# quick peek
for k, v in dfs.items():
    print(k, v.head(3))

Saved ../data/raw/api_source-coinbase_symbol-BTCUSD_window-20250523_to_20250821_granularity-1h_20250821-150131.csv
Saved ../data/raw/api_source-coinbase_symbol-ETHUSD_window-20250523_to_20250821_granularity-1h_20250821-150140.csv
BTC-USD                        time        low       high       open      close  \
0 2025-05-23 19:00:00+00:00  108626.43  109222.20  108822.72  108747.25   
1 2025-05-23 20:00:00+00:00  108205.69  108755.59  108744.54  108261.23   
2 2025-05-23 21:00:00+00:00  108078.86  108700.00  108261.24  108320.71   

       volume  
0  970.097006  
1  452.581600  
2  225.371478  
ETH-USD                        time      low     high     open    close       volume
0 2025-05-23 19:00:00+00:00  2556.54  2576.15  2560.35  2560.99  8207.541172
1 2025-05-23 20:00:00+00:00  2527.00  2563.32  2561.09  2543.89  6278.149993
2 2025-05-23 21:00:00+00:00  2527.92  2554.12  2543.56  2535.03  2905.645630


## Part 2 — Scrape a Public Table (Required)
Replace `SCRAPE_URL` with a permitted page containing a simple table.

In [16]:
# Public, permitted page with a simple table
SCRAPE_URL = 'https://en.wikipedia.org/wiki/List_of_cryptocurrencies'  # permitted page
headers = {'User-Agent': 'AFE-Homework/1.0'}

try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30); resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')  # match starter parser

    # Parse with BeautifulSoup; build DataFrame
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]

    # Keep only rows that match header length (prevents shape errors if page has multiple tables)
    data = [r for r in data if len(r) == len(header)]

    df_scrape = pd.DataFrame(data, columns=header)

except Exception as e:
    print('Scrape failed, using inline demo table:', e)
    html = '<table><tr><th>Ticker</th><th>Price</th></tr><tr><td>AAA</td><td>101.2</td></tr></table>'
    soup = BeautifulSoup(html, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)

# Validate numeric/text columns
v_scrape = validate(df_scrape, list(df_scrape.columns)); v_scrape

# Save raw CSV to data/raw/
_ = save_csv(df_scrape, prefix='scrape', source='wikipedia', page='cryptocurrencies')

Saved ../data/raw/scrape_source-wikipedia_page-cryptocurrencies_20250821-145540.csv


## Documentation

### API Source
- **URL/endpoint:** `GET https://api.exchange.coinbase.com/products/{product_id}/candles`
- **Symbols:** `BTC-USD`, `ETH-USD`
- **Params used:** `start` (ISO UTC), `end` (ISO UTC), `granularity=3600` (1h)
- **Validation run:** required columns `['time','open','high','low','close','volume']`; numeric coercion for OHLCV; non-empty shape; total NAs = 0; timestamps ascending
- **Saved via:** `save_csv(..., prefix='api', source='coinbase', symbol=<BTCUSD|ETHUSD>, window=<YYYYMMDD_to_YYYYMMDD>, granularity='1h')`

### Scrape Source
- **URL:** `https://en.wikipedia.org/wiki/List_of_cryptocurrencies`
- **Table description:** first suitable `wikitable`; header from first non-empty row; kept rows matching header length
- **Validation run:** passed all columns to `validate(...)`; confirmed non-empty table and consistent row lengths; no numeric coercion applied because columns are text-only
- **Saved via:** `save_csv(df_scrape, prefix='scrape', source='wikipedia', page='cryptocurrencies')`

### Assumptions & Risks
- **API:** per-request max of ~300 candles handled via time-window iteration; possible schema or rate-limit changes; data treated as UTC
- **Scrape:** page structure may change; header detection could shift if tables reorder

### `.env` Confirmation
- Coinbase candles require no key; `.env` (if used later) is **not committed** (`.env`, `.env.*` in `.gitignore`)